** In this Jupyter Notebook, the absolute + sensitivity analysis results are generated for a One-at-a-time sensitivity analysis of fleet share percentage for the proliferation risk evaluation metric. The reason why this metric is separated from the others is that you need the -exp version of each sqlite and pyne on your machine to make it work **

In [1]:
import cymetric as cym
from cymetric import timeseries
import matplotlib as plt 
import pandas as pd
import numpy as np
import sys 
sys.path.insert(0, '../../../scripts/')
import output as oup

/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/Users/gwenchee/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


# This is when you already have the results 

In [2]:
df_p = pd.read_csv('fs-df-pu.csv',index_col='FS')
df_p

,Max Pu in all CP,Pu Quality in all CP at Max Pu,Max Pu in HLW,Pu Quality in HLW at Max Pu,Max Pu in all RPR,Pu Quality in all RPR at Max Pu
FS,,,,,,
0,256794.0273,0.656144,45816.01856,0.616190,2357604.009,0.548165
5,273733.1322,0.618838,44338.72473,0.617842,2461421.306,0.512185
10,279694.7846,0.605865,44055.53671,0.618561,2513133.758,0.496724
15,285794.8482,0.593123,42905.59368,0.619136,2552142.815,0.480625
20,291786.5398,0.581150,43133.60862,0.619916,2605589.017,0.466663


# The rest of the code below is to generate the above results

In [ ]:
starter_string = 'FS'
scenario_nums = ['0','5','10','15','20']

In [ ]:
df_p = oup.initialize_df(scenario_index=starter_string,
                                 scenarios_nums=scenario_nums)

In [ ]:
df_p['Max Pu in all CP'] = 0
df_p['Pu Quality in all CP at Max Pu'] = 0
df_p['Max Pu in HLW'] = 0
df_p['Pu Quality in HLW at Max Pu'] = 0
df_p['Max Pu in all RPR'] = 0
df_p['Pu Quality in all RPR at Max Pu'] = 0

In [ ]:
output_start = '../cyclus-files/oat/fleet-share/fs'
ev_dict = {}
for x in range(len(scenario_nums)): 
    output_file = output_start + scenario_nums[x]+'-exp.sqlite'
    ev_dict[scenario_nums[x]] = cym.Evaluator(db=cym.dbopen(output_file),write=True)

In [ ]:
for x in range(len(scenario_nums)): 
    cp = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_cp = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrstorage','moxstorage','frstorage'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in all CP'] = cp.max()
    df_p.loc[scenario_nums[x],'Pu Quality in all CP at Max Pu'] = fissile_cp[cp.idxmax()]/cp.max()
    hlw = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_hlw = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['enrichmentsink','lwrsink','moxsink','frsink'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in HLW'] = hlw.max()
    df_p.loc[scenario_nums[x],'Pu Quality in HLW at Max Pu'] = fissile_hlw[hlw.idxmax()]/hlw.max()
    rpr = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-238','pu-239','pu-240','pu-241','pu-242','pu-244'])['Quantity']
    fissile_rpr = cym.timeseries.inventories(ev_dict[scenario_nums[x]],facilities=['lwrreprocessing','moxreprocessing','frreprocessing'],nucs=['pu-239','pu-241'])['Quantity']
    df_p.loc[scenario_nums[x],'Max Pu in all RPR'] = rpr.max()
    df_p.loc[scenario_nums[x],'Pu Quality in all RPR at Max Pu'] = fissile_rpr[rpr.idxmax()]/rpr.max()

In [ ]:
df_p.to_csv('fs-df-pu.csv')

In [6]:
df_p_sa = oup.sensitivity(15,df_p)
df_p_sa

,Max Pu in all CP,Pu Quality in all CP at Max Pu,Max Pu in HLW,Pu Quality in HLW at Max Pu,Max Pu in all RPR,Pu Quality in all RPR at Max Pu
FS,,,,,,
0,-10.147426,10.625314,6.783323,-0.475960,-7.622567,14.052645
5,-4.220411,4.335579,3.340196,-0.209126,-3.554719,6.566601
10,-2.134420,2.148294,2.680170,-0.092899,-1.528483,3.349693
15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20,2.096501,-2.018603,0.531434,0.125945,2.094170,-2.904799


In [5]:
df_p_sa.to_csv('fs-df-pu-sa.csv')